# Sample Submission Extra Images
Based on the sample code provided by [DataTalks.Club](https://github.com/DataTalksClub/kitchenware-competition-starter/blob/main/keras-starter.ipynb) to use as a starter notebook and make an initial submission.

Uses the [extra images](https://www.kaggle.com/datasets/agrigorev/kitchenware-extra-images) provided for the competition

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras

2023-01-11 18:55:31.890535: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data_dir = '/run/user/1000/gvfs/smb-share:server=titan.local,share=datasets/kitchenware-classification'

In [3]:
df_train_full = pd.read_csv(data_dir+'/train.csv', dtype={'Id': str})
df_extra = pd.read_csv(data_dir+'/data.csv', dtype={'Id': str})
df_train_full = pd.concat([df_train_full, df_extra], ignore_index = True)
df_train_full['filename'] = data_dir+'/images/' + df_train_full['Id'] + '.jpg'
df_train_full.head()

,Id,label,filename
0,0560,glass,/run/user/1000/gvfs/smb-share:server=titan.loc...
1,4675,cup,/run/user/1000/gvfs/smb-share:server=titan.loc...
2,0875,glass,/run/user/1000/gvfs/smb-share:server=titan.loc...
3,4436,spoon,/run/user/1000/gvfs/smb-share:server=titan.loc...
4,8265,plate,/run/user/1000/gvfs/smb-share:server=titan.loc...


In [4]:
val_cutoff = int(len(df_train_full) * 0.8)
df_train = df_train_full[:val_cutoff]
df_val = df_train_full[val_cutoff:]

In [5]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_dataframe(
    df_train,
    x_col='filename',
    y_col='label',
    target_size=(150, 150),
    batch_size=32,
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_generator = val_datagen.flow_from_dataframe(
    df_val,
    x_col='filename',
    y_col='label',
    target_size=(150, 150),
    batch_size=32,
)

Found 6390 validated image filenames belonging to 6 classes.
Found 1598 validated image filenames belonging to 6 classes.


In [7]:
base_model = Xception(
    weights='imagenet',
    include_top=False,
    input_shape=(150, 150, 3)
)
base_model.trainable = False

inputs = keras.Input(shape=(150, 150, 3))

base = base_model(inputs, training=False)
vectors = keras.layers.GlobalAveragePooling2D()(base)
outputs = keras.layers.Dense(6)(vectors)

model = keras.Model(inputs, outputs)

2023-01-11 18:55:45.503169: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-11 18:55:45.546048: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-11 18:55:45.546223: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-11 18:55:45.546582: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

In [8]:
learning_rate = 0.01
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

loss = keras.losses.CategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [9]:
history = model.fit(
    train_generator,
    epochs=2,
    validation_data=val_generator
)

Epoch 1/2


2023-01-11 18:55:50.037883: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401


  1/200 [..............................] - ETA: 11:41 - loss: 1.9519 - accuracy: 0.0625

2023-01-11 18:55:50.897790: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-01-11 18:55:50.898324: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-01-11 18:55:50.898333: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-01-11 18:55:50.898820: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-01-11 18:55:50.898856: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2023-01-11 18:55:51.010539: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged onc

200/200 [==============================] - 132s 648ms/step - loss: 0.6928 - accuracy: 0.8360 - val_loss: 0.6749 - val_accuracy: 0.8698
Epoch 2/2
200/200 [==============================] - 140s 700ms/step - loss: 0.4460 - accuracy: 0.8997 - val_loss: 0.6856 - val_accuracy: 0.8686


In [10]:
df_test = pd.read_csv(data_dir+'/test.csv', dtype={'Id': str})
df_test['filename'] = data_dir+'/images/' + df_test['Id'] + '.jpg'
df_test.head()

,Id,filename
0,0678,/run/user/1000/gvfs/smb-share:server=titan.loc...
1,3962,/run/user/1000/gvfs/smb-share:server=titan.loc...
2,9271,/run/user/1000/gvfs/smb-share:server=titan.loc...
3,5133,/run/user/1000/gvfs/smb-share:server=titan.loc...
4,8842,/run/user/1000/gvfs/smb-share:server=titan.loc...


In [11]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = test_datagen.flow_from_dataframe(
    df_test,
    x_col='filename',
    class_mode='input',
    target_size=(150, 150),
    batch_size=32,
    shuffle=False
)

Found 3808 validated image filenames.


In [12]:
y_pred = model.predict(test_generator)

119/119 [==============================] - 65s 542ms/step


In [13]:
classes = np.array(list(train_generator.class_indices.keys()))
classes

array(['cup', 'fork', 'glass', 'knife', 'plate', 'spoon'], dtype='<U5')

In [14]:
predictions = classes[y_pred.argmax(axis=1)]

In [15]:
df_submission = pd.DataFrame()
df_submission['filename'] = test_generator.filenames
df_submission['label'] = predictions

df_submission['Id'] = df_submission.filename.str[len(data_dir+'/images/'):-4]
del df_submission['filename']

In [16]:
df_submission[['Id', 'label']].to_csv('submissions/sample_submission_extra.csv', index=False)

In [17]:
!kaggle competitions submit kitchenware-classification -f submissions/sample_submission_extra.csv -m 'validation: 0.8686'

100%|██████████████████████████████████████| 39.1k/39.1k [00:01<00:00, 20.3kB/s]
Successfully submitted to Kitchenware Classification